In [1]:
from PIL import Image
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import utils
from tqdm import tqdm
import warnings
import torchvision.models as models

warnings.simplefilter('ignore', Image.DecompressionBombWarning)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
test_path = "/DAS_Storage4/hyungseok/Validation"

In [3]:
device = torch.device("cuda:1")

In [4]:
test_trans = transforms.Compose([                          
                           transforms.ToTensor(),
                           transforms.Resize((224,224)),                                              
                           transforms.Normalize((0.4455003,0.530581,0.30614495),(0.17718968,0.1856016,0.16673586))                           
                           ])

testset = torchvision.datasets.ImageFolder(root = test_path,
                                           transform = test_trans)

test_loader = DataLoader(testset, batch_size = 128, shuffle = False, num_workers = 8)

In [5]:
model = models.resnet50(pretrained=False)
num_feature = model.fc.in_features
model.fc = nn.Linear(num_feature, 128)

In [6]:
model = model
utils.init_weights(model, init_type='uniform')
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas = (0.9, 0.999), eps = 1e-9, weight_decay = 1e-5)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
  
    model = nn.DataParallel(model, device_ids=[1,2,3,4])

model.to(device)

Let's use 7 GPUs!


DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
      

In [7]:
checkpoint = torch.load('model/resnet/41-final_resnet_model.pt')

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [8]:
total_pred = []
with torch.no_grad():
        model.eval()

        for idx, (x, y) in tqdm(enumerate(test_loader)):

            x, y = x.to(device), y.to(device)

            valid_output = model(x)
            _, predicted = torch.max(valid_output.data, 1)
            
            total_pred.append(predicted.detach().cpu().numpy())
            
total_pred = np.concatenate(total_pred)
            

518it [18:42,  2.17s/it]


In [9]:
prediction = pd.read_csv('data/prediction_example.csv')
prediction['pred'] = pd.DataFrame(total_pred)
prediction.to_csv('data/pred.csv', index = None)